In [ ]:
 from google.colab import drive
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Load the dataset
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/Datasets/Emotion.csv'
df = pd.read_csv(dataset_path)

Mounted at /content/drive


In [ ]:
classes_to_keep = ["happy", "sadness", "fear", "confusion"]

# Filter the DataFrame to include only the specified classes
df = df[df['Emotion'].isin(classes_to_keep)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15976 entries, 0 to 21498
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     15976 non-null  object
 1   Emotion  15976 non-null  object
dtypes: object(2)
memory usage: 374.4+ KB


In [ ]:
from scipy import stats

# Count the occurrences of each emotion label
emotion_counts = df['Emotion'].value_counts()

# Set a threshold for the minimum occurrence count below which data points are considered outliers
threshold = 5

# Identify the emotions that occur less frequently than the threshold
outlier_emotions = emotion_counts[emotion_counts < threshold].index

# Remove the outliers from the dataset
clean_df = df[~df['Emotion'].isin(outlier_emotions)]

# Verify the resulting dataset
print(clean_df.head())

                                                Text  Emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
6  ive been feeling a little burdened lately wasn...  sadness
8  i feel as confused about life as a teenager or...     fear
9  i have been with petronas for years i feel tha...    happy


In [ ]:
df['Emotion'] = df['Emotion'].replace(['fear', 'confusion'], 'anxiety')

X = df["Text"]
y = df["Emotion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = df['Emotion'], test_size=0.2, random_state=42)

In [ ]:
df.columns

Index(['Text', 'Emotion'], dtype='object')

In [ ]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Convert the text data to numerical representation
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100)

# Train the model
model.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [ ]:
!pip install joblib

In [ ]:
import joblib

# Assuming 'rf_model' is your trained Random Forest model
model_filename = '/content/drive/My Drive/models/text_model.pkl'
joblib.dump(model, model_filename)

['/content/drive/My Drive/models/text_model.pkl']

In [ ]:
tfidf_vectorizer_filename = '/content/drive/My Drive/models/vectorizer.pkl'
joblib.dump(vectorizer, tfidf_vectorizer_filename)

['/content/drive/My Drive/models/vectorizer.pkl']

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Calculate and print the accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9167709637046307


In [ ]:
from google.colab import drive
import joblib
from sklearn.preprocessing import normalize

# Mount Google Drive
drive.mount('/content/drive')

def predict_depression():
    # Load the TF-IDF vectorizer
    tfidf_vectorizer_filename = '/content/drive/MyDrive/models/vectorizer.pkl'
    vectorizer = joblib.load(tfidf_vectorizer_filename)

    # Load the text classification model (trained on the filtered dataset)
    text_model_filename = '/content/drive/MyDrive/models/text_model.pkl'
    text_model = joblib.load(text_model_filename)

    # Define the list of emotion labels based on your filtered dataset
    emotion_labels = ["happy", "sadness", "anxiety"]

    # Initialize an empty list to store user responses
    user_responses = []

    # Ask 5 questions
    questions = [
        "How would you describe your current emotional state in a few words?",
        "What thoughts or situations are on your mind right now that might be affecting your mood?",
        "Is there anything bringing you joy or happiness at this moment?",
        "Do you feel any unease or sadness right now? If yes, can you share what might be contributing to it?",
        "In a few words, how would you describe the atmosphere around you at this very moment?"
    ]

    for i, question in enumerate(questions, start=1):
        user_input = input(f"Question {i}: {question}\nYour Answer: ")
        user_responses.append(user_input)

    # Transform user responses using the loaded vectorizer
    user_responses_tfidf = vectorizer.transform(user_responses)

    # Make predictions on user responses
    predicted_probabilities = text_model.predict_proba(user_responses_tfidf)

    # Normalize the probabilities
    normalized_probabilities = normalize(predicted_probabilities, norm='l1', axis=1)

    # Calculate the average predicted percentage for "sadness"
    average_anx_percentage = sum(normalized_probabilities[:, 2]) / len(normalized_probabilities)

    return f"Average Normalized Predicted Percentage for 'Anxiety': {average_anx_percentage * 100:.2f}%"

result = predict_depression()
print(result)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Question 1: How would you describe your current emotional state in a few words?
Your Answer: asdfgh
Question 2: What thoughts or situations are on your mind right now that might be affecting your mood?
Your Answer: sdfhjk
Question 3: Is there anything bringing you joy or happiness at this moment?
Your Answer: sdfhk
Question 4: Do you feel any unease or sadness right now? If yes, can you share what might be contributing to it?
Your Answer: rtdfj
Question 5: In a few words, how would you describe the atmosphere around you at this very moment?
Your Answer: asdfgjhkj
Average Normalized Predicted Percentage for 'Anxiety': 33.00%
